# Домашнее задание по теме «Коллаборативная фильтрация»
ПАКЕТ SURPRISE

- используйте данные MovieLens 1M
- можно использовать любые модели из пакета
- получите RMSE на тестовом сете **0.87 и ниже**

*Комментарий преподавателя* :   
В ДЗ на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе CrossValidation (5 фолдов), а не отложенном датасете.

In [1]:
# Загружаем необходимые библиотеки
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise import SVD

import pandas as pd

In [2]:
# Загружаем данные
movies = pd.read_csv('movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
# Загружаем данные
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
# Объединяем данные фильмов и рейтинга
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


In [5]:
# Составляем датасет для модели
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})
dataset.head()

,uid,iid,rating
0,1.0,Toy Story (1995),4.0
1,5.0,Toy Story (1995),4.0
2,7.0,Toy Story (1995),4.5
3,15.0,Toy Story (1995),2.5
4,17.0,Toy Story (1995),4.5


In [6]:
# Определяем минимальный рейтинг
ratings.rating.min()

0.5

In [7]:
# Определяем максимальный рейтинг
ratings.rating.max()

5.0

In [8]:
# Определяем ридер и задаём масштаб значений
reader = Reader(rating_scale=(0.5, 5.0))

In [9]:
# Формируем датасет
data = Dataset.load_from_df(dataset, reader)

In [10]:
# Разделяем данные не тренировочную и тестовую части
trainset, testset = train_test_split(data, test_size=.15)

In [11]:
# Обучаем модель KNN
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [12]:
# Осуществляем предсказание
test_pred = algo.test(testset)

In [13]:
# Определяем метрику RMSE
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8886


0.8885567024383394

Требуется улучшить качество модели.

In [14]:
# Обучаем модель KNN
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [15]:
# Осуществляем предсказание
test_pred = algo.test(testset)

In [16]:
# Определяем метрику RMSE
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8751


0.8750504339085677

Качество модели недостаточно. Используем CrossValidation.

In [17]:
algo = SVD(n_epochs = 25)
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8638  0.8802  0.8753  0.8678  0.8805  0.8735  0.0067  
Fit time          4.69    4.74    4.66    4.63    4.71    4.68    0.04    
Test time         0.09    0.09    0.14    0.16    0.09    0.12    0.03    


{'test_rmse': array([0.86382672, 0.88022285, 0.87529878, 0.86784923, 0.88046744]),
 'fit_time': (4.690242767333984,
  4.744898080825806,
  4.656749963760376,
  4.625749826431274,
  4.705759286880493),
 'test_time': (0.09375619888305664,
  0.09375596046447754,
  0.14063429832458496,
  0.15626096725463867,
  0.09376716613769531)}

Качество модели всё ещё недостаточно. Сменим модель.

In [18]:
# Обучаем модел
algo = SVD(n_epochs = 25)
algo.fit(trainset)

In [19]:
# Осуществляем предсказание
test_pred = algo.test(testset)

In [20]:
# Определяем метрику RMSE
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8691


0.8690590062752487